In [1]:
def rename_columns_containing_substrings(df, substrings_replacements):
    for substring, new_name in substrings_replacements.items():
        df.columns = [new_name if substring.lower() in col.lower() else col for col in df.columns]
    return df

In [2]:
def combine_columns_containing_substrings(df, substrings_new_columns):
    for substring, new_column_name in substrings_new_columns.items():
        # Identify columns that contain the substring
        columns_to_combine = [col for col in df.columns if substring.lower() in col.lower()]
        
        # Combine the values of these columns into a single column
        df[new_column_name] = df[columns_to_combine].apply(lambda row: ', '.join(row.dropna().astype(str)), axis=1)
        
        # Drop the original columns
        df.drop(columns_to_combine, axis=1, inplace=True)
    
    return df

In [3]:
def drop_columns_containing_substrings(df, substrings):
    
    # Start with the original DataFrame
    df_dropped = df.copy()
    
    # Iterate over the list of substrings
    for substring in substrings:
        # Find columns that contain the substring
        cols_to_drop = [col for col in df_dropped.columns if substring.lower() in col.lower()]
        # Drop these columns
        df_dropped.drop(cols_to_drop, axis=1, inplace=True)
    
    return df_dropped

In [4]:
def standardize_terms(df, term_mapping, exclude_terms=None):
    # Make a copy of the DataFrame to avoid changing the original data
    df_standardized = df.copy()
    
    exclude_terms = [term.lower() for term in exclude_terms]

    # Iterate over each term and its standard replacement
    for term_to_find, standard_term in term_mapping.items():
        # Apply the standardization to each element in the DataFrame
        df_standardized = df_standardized.applymap(lambda x: standard_term if (term_to_find.lower() in str(x).lower() and (exclude_terms is None or str(x).lower() not in exclude_terms)) else x)
    
    return df_standardized

In [5]:
# Define your renaming dictionary
rename_columns={
' age': 'Age',
'gender': 'Gender',
'country': 'Country',
'education': 'Education',
'title': 'Job',
'years have you been writing code': 'Years_of_programming',
'Approximately how many times have you used a TPU': 'Frequency_of_using TPU',
'years have you used machine learning methods': 'Years_of_machine_learning',
'size of the company': 'Company_size',
'yearly compensation': 'Yearly_compensation'
}

#These are the features may not help doing the job recommendation
meaningless_features = ['recommend', 'how many individuals are responsible for data science workloads at your place of business',
                     'Does your current employer incorporate machine learning methods into their business',
                     'spent on machine learning', 'in the next 2 years']

#These are the features in 2021 dataset, but not in other datasets
_2021features = ['In what industry is your current employer/contract', 'primary tool','Of the cloud platforms that you are familiar with, which has the best developer experience', 
               'Do you use any of the following data storage products', 'computing platform do you use most often', 'publicly share']

#These are the features in 2022 dataset, but not in other datasets
_2022features = ['Did your research make use of machine learning', 'What products or platforms did you find to be most helpful when you first started studying data science', 
                 'Have you ever published any academic research','Have you ever published any academic research','Do you download pre-trained model weights',
                 'Which of the following ML model hubs/repositories do you use most often', 
                 'big data products (relational database, data warehouse, data lake, or similar) do you use most often',
                 'business intelligence tools do you use most often', 'Do you use any of the following products to serve your machine learning models','ethical AI products']

substrings_to_drop = meaningless_features + _2021features + _2022features

# Dictionary of substrings and their corresponding new column names
substrings_new_columns = {
    'programming languages': 'Language',
    'integrated development environments': 'IDE',
    'hosted notebook products': 'Notebook',
    'hardware': 'Hardware',
    'Visualization': 'Visualization_tool',
    'machine learning frameworks': 'ML_Framework',
    'ML algorithms': 'ML_algorithm',
    'computer vision': 'Computer_Vision_Methods',
    'NLP': 'NLP_Methods',
    'any activities': 'Related_activities',
    'cloud computing platforms': 'Cloud_computing_platforms',
    'cloud computing products': 'Cloud_computing_products',
    'machine learning products': 'ML_Products',
    'data products': 'Big_data_products',
    'business intelligence tools': 'Bussiness_intelligence tools',
    'automated machine learning tools': 'AutoML_tools',
    'any tools to help': 'ML_tools',
    'data science courses': 'Course_platform',
    'favorite media sources': 'Media_sources'
     }

# Define your term mapping dictionary for standardization
term_mapping = {'Jupyter': 'Jupyter',
                'Amazon Sagemaker Studio': 'AmazonSagemakerStudio',
                'Data Analyst': 'DataAnalyst',
                'Manager': 'Product/ProjectManager',
                 'Machine Learning': 'MachineLearningEngineer'}
exclude_terms = ['Binder / JupyterHub']

In [6]:
# Create a function that encapsulates the entire preprocessing pipeline
def data_preprocessing_pipeline(df):
    # Perform the preprocessing steps in order
    df = rename_columns_containing_substrings(df, rename_columns)
    df = drop_columns_containing_substrings(df, substrings_to_drop)
    df = standardize_terms(df, term_mapping, exclude_terms)
    df = combine_columns_containing_substrings(df, substrings_new_columns)
    return df

In [7]:
import pandas as pd
import numpy as np
import function as func
from sklearn.preprocessing import MultiLabelBinarizer
import re

2023-11-11 05:15:32.235 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-11-11 05:15:32.237 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [8]:
# Load the datasets
file_paths = {
    2020: 'kaggle_survey_2020_responses.csv',
    2021: 'kaggle_survey_2021_responses.csv',
    2022: 'kaggle_survey_2022_responses.csv'
}

datasets = {year: pd.read_csv(file_path, low_memory=False) for year, file_path in file_paths.items()}

In [9]:
processed_datasets = {}
for year, dataset in datasets.items():
    # The first row contains the column names
    header = datasets[year].iloc[0]
    # The data starts from the second row
    new_dataset = pd.DataFrame(datasets[year].values[1:], columns=header)
    # Drop the first column, which is survey duration
    new_dataset = new_dataset.drop(new_dataset.columns[0], axis=1)
    # Process the data by going through the pipeline
    processed_datasets[year] = data_preprocessing_pipeline(new_dataset)
    # Replace white spaces with NaN in the new dataset
    processed_datasets[year].replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [10]:
#Add student to the job column if the participants are students in 2022 dataset
processed_datasets[2022]['Job'] = processed_datasets[2022].apply(lambda row: 'Student' if row['Are you currently a student? (high school, university, or graduate)'] == 'Yes' else row['Job'], axis=1)

# Drop the "Are you currently a student?" column
processed_datasets[2022].drop(columns=['Are you currently a student? (high school, university, or graduate)'], inplace=True)

In [11]:
combined_dataset = pd.concat(processed_datasets.values(), axis=0)
#drop the participants if job is na value
combined_dataset = combined_dataset.dropna(subset=['Job'])
combined_dataset.reset_index(drop=True, inplace=True)
combined_dataset

,Age,Gender,Country,Education,Job,Years_of_programming,Frequency_of_using TPU,Years_of_machine_learning,Company_size,Yearly_compensation,...,Related_activities,Cloud_computing_platforms,Cloud_computing_products,ML_Products,Big_data_products,Bussiness_intelligence tools,AutoML_tools,ML_tools,Course_platform,Media_sources
0,35-39,Man,Colombia,Doctoral degree,Student,5-10 years,2-5 times,1-2 years,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Coursera, Kaggle Learn Courses, University Cou...","Kaggle (notebooks, forums, etc), Journal Publi..."
1,30-34,Man,United States of America,Master’s degree,Data Engineer,5-10 years,2-5 times,1-2 years,"10,000 or more employees","100,000-124,999",...,Analyze and understand data to influence produ...,"Amazon Web Services (AWS) , Microsoft Azure ...","Amazon EC2 , AWS Lambda , Azure Functions ,...",Amazon SageMaker,"PostgresSQL , Amazon Redshift , Amazon Athena","Amazon QuickSight, Microsoft Power BI, Tableau",No / None,No / None,"Coursera, DataCamp, Udemy","Twitter (data science influencers), Reddit (r/..."
2,35-39,Man,Argentina,Bachelor’s degree,Software Engineer,10-20 years,Never,MachineLearningEngineer,"1000-9,999 employees","15,000-19,999",...,None of these activities are an important part...,None,NaN,NaN,MySQL,None,No / None,No / None,"Coursera, edX, Udacity, Udemy","Email newsletters (Data Elixir, O'Reilly Data ..."
3,30-34,Man,United States of America,Master’s degree,Data Scientist,5-10 years,2-5 times,3-4 years,250-999 employees,"125,000-149,999",...,Analyze and understand data to influence produ...,"Amazon Web Services (AWS) , Salesforce Cloud","Amazon EC2 , AWS Lambda",No / None,"MySQL , Amazon Redshift",Tableau,No / None,No / None,"Coursera, DataCamp, LinkedIn Learning, Univers...",None
4,30-34,Man,Japan,Master’s degree,Software Engineer,3-5 years,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67836,22-24,Man,United States of America,Master’s degree,Student,3-5 years,NaN,1-2 years,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"Kaggle (notebooks, forums, etc), YouTube (Kagg..."
67837,60-69,Man,United States of America,Bachelor’s degree,Student,20+ years,NaN,2-3 years,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,University Courses (resulting in a university ...,"Email newsletters (Data Elixir, O'Reilly Data ..."
67838,25-29,Man,Turkey,Master’s degree,Engineer (non-software),< 1 years,Never,Under 1 year,"10,000 or more employees","25,000-29,999",...,None of these activities are an important part...,Microsoft Azure,No / None,No / None,Microsoft Azure SQL Database,Tableau,No / None,No / None,"DataCamp, Udemy","Twitter (data science influencers), Kaggle (no..."
67839,35-39,Woman,Israel,Doctoral degree,Research Scientist,3-5 years,NaN,Under 1 year,0-49 employees,"15,000-19,999",...,None of these activities are an important part...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Kaggle Learn Courses, Udacity, University Cour...","YouTube (Kaggle YouTube, Cloud AI Adventures, ..."


In [12]:
combined_dataset.to_csv('preprocessed_dataset.csv', index = False)

In [13]:
combined_dataset.isna().sum()

Age                                 0
Gender                              0
Country                             0
Education                         367
Job                                 0
Years_of_programming              609
Frequency_of_using TPU          22065
Years_of_machine_learning        8339
Company_size                    31121
Yearly_compensation             33585
Language                         4918
IDE                              5667
Notebook                         6403
Hardware                        21751
Visualization_tool               8192
ML_Framework                    18228
ML_algorithm                    19000
Computer_Vision_Methods         47754
NLP_Methods                     54400
Related_activities              32659
Cloud_computing_platforms       45957
Cloud_computing_products        53307
ML_Products                     49071
Big_data_products               46382
Bussiness_intelligence tools    46536
AutoML_tools                    47177
ML_tools    

In [14]:
combined_dataset = combined_dataset[(combined_dataset['Job'] != 'Student') & (combined_dataset['Job'] != 'Other') & (combined_dataset['Job'] != 'Currently not employed')]

In [15]:
combined_dataset['Job'].unique()

array(['Data Engineer', 'Software Engineer', 'Data Scientist',
       'DataAnalyst', 'Research Scientist', 'Statistician',
       'Product/ProjectManager', 'MachineLearningEngineer',
       'Business Analyst', 'DBA/Database Engineer',
       'Developer Relations/Advocacy', 'Developer Advocate',
       'Engineer (non-software)', 'Teacher / professor',
       'Data Administrator', 'Data Architect'], dtype=object)

In [16]:
combined_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33951 entries, 1 to 67839
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Age                           33951 non-null  object
 1   Gender                        33951 non-null  object
 2   Country                       33951 non-null  object
 3   Education                     33951 non-null  object
 4   Job                           33951 non-null  object
 5   Years_of_programming          33862 non-null  object
 6   Frequency_of_using TPU        28036 non-null  object
 7   Years_of_machine_learning     31112 non-null  object
 8   Company_size                  32064 non-null  object
 9   Yearly_compensation           29862 non-null  object
 10  Language                      32123 non-null  object
 11  IDE                           31870 non-null  object
 12  Notebook                      31670 non-null  object
 13  Hardware        

In [17]:
combined_dataset.isnull().sum()

Age                                 0
Gender                              0
Country                             0
Education                           0
Job                                 0
Years_of_programming               89
Frequency_of_using TPU           5915
Years_of_machine_learning        2839
Company_size                     1887
Yearly_compensation              4089
Language                         1828
IDE                              2081
Notebook                         2281
Hardware                         5819
Visualization_tool               2835
ML_Framework                     6725
ML_algorithm                     6902
Computer_Vision_Methods         22456
NLP_Methods                     25633
Related_activities               3252
Cloud_computing_platforms       14331
Cloud_computing_products        20540
ML_Products                     16958
Big_data_products               14713
Bussiness_intelligence tools    14843
AutoML_tools                    15405
ML_tools    

In [18]:
null_columns = ['Frequency_of_using TPU','Computer_Vision_Methods','NLP_Methods',
                                                                    'Cloud_computing_platforms','Cloud_computing_products','ML_Products','Big_data_products','Bussiness_intelligence tools',
                                                                       'AutoML_tools', 'ML_tools']
combined_dataset = drop_columns_containing_substrings(combined_dataset, null_columns)
combined_dataset.reset_index(drop=True, inplace=True)

In [19]:
#Convert data in range to average integer
combined_dataset['Age'] = combined_dataset['Age'].apply(func.convert_range_to_average)
combined_dataset['Yearly_compensation'] = combined_dataset['Yearly_compensation'].apply(func.convert_range_to_average)

In [20]:
combined_dataset

,Age,Gender,Country,Education,Job,Years_of_programming,Years_of_machine_learning,Company_size,Yearly_compensation,Language,IDE,Notebook,Hardware,Visualization_tool,ML_Framework,ML_algorithm,Related_activities,Course_platform,Media_sources
0,32,Man,United States of America,Master’s degree,Data Engineer,5-10 years,1-2 years,"10,000 or more employees",112500.0,"Python, R, SQL","Visual Studio, PyCharm , Sublime Text",Colab Notebooks,GPUs,"Matplotlib , Seaborn , Ggplot / ggplot2 , ...","Scikit-learn , TensorFlow , Keras , PyTo...","Linear or Logistic Regression, Convolutional N...",Analyze and understand data to influence produ...,"Coursera, DataCamp, Udemy","Twitter (data science influencers), Reddit (r/..."
1,38,Man,Argentina,Bachelor’s degree,Software Engineer,10-20 years,MachineLearningEngineer,"1000-9,999 employees",17500.0,"Java, Javascript, Bash","Visual Studio Code (VSCode), Notepad++ , ...",None,None,D3 js,NaN,NaN,None of these activities are an important part...,"Coursera, edX, Udacity, Udemy","Email newsletters (Data Elixir, O'Reilly Data ..."
2,32,Man,United States of America,Master’s degree,Data Scientist,5-10 years,3-4 years,250-999 employees,137500.0,"Python, SQL, Bash",PyCharm,None,None,"Matplotlib , Seaborn , Plotly / Plotly Expr...","Scikit-learn , TensorFlow , Keras , Xgbo...","Linear or Logistic Regression, Decision Trees ...",Analyze and understand data to influence produ...,"Coursera, DataCamp, LinkedIn Learning, Univers...",None
3,32,Man,Japan,Master’s degree,Software Engineer,3-5 years,NaN,NaN,NaN,Python,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,32,Man,India,Bachelor’s degree,DataAnalyst,< 1 years,NaN,NaN,NaN,"Python, R","Jupyter, RStudio",None,None,"Matplotlib , Seaborn , Ggplot / ggplot2",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33946,42,Man,United States of America,Master’s degree,Data Architect,10-20 years,Under 1 year,"10,000 or more employees",112500.0,"Python, SQL, C#, Java, Javascript, Bash","Jupyter, Visual Studio Code (VSCode) , PyCha...","Kaggle Notebooks, Colab Notebooks, Google Clo...",NaN,"Matplotlib , Seaborn , Plotly / Plotly Expr...","Scikit-learn , TensorFlow , Xgboost","Linear or Logistic Regression, Decision Trees ...",Analyze and understand data to influence produ...,"Kaggle Learn Courses, DataCamp, LinkedIn Learn...","Twitter (data science influencers), Kaggle (no..."
33947,38,Man,India,Master’s degree,Data Scientist,1-3 years,1-2 years,"10,000 or more employees",2500.0,Python,Jupyter,None,None,"Matplotlib , Seaborn","Scikit-learn , Xgboost","Linear or Logistic Regression, Decision Trees ...",Build and/or run the data infrastructure that ...,Other,"YouTube (Kaggle YouTube, Cloud AI Adventures, ..."
33948,42,Man,India,Bachelor’s degree,Data Engineer,1-3 years,MachineLearningEngineer,0-49 employees,500.0,"Python, Javascript","Jupyter, Visual Studio Code (VSCode) , Note...",None,NaN,"Matplotlib , Seaborn",NaN,NaN,Analyze and understand data to influence produ...,"DataCamp, Udemy","YouTube (Kaggle YouTube, Cloud AI Adventures, ..."
33949,28,Man,Turkey,Master’s degree,Engineer (non-software),< 1 years,Under 1 year,"10,000 or more employees",27500.0,"Python, SQL","PyCharm , Spyder , Jupyter",None,None,Seaborn,"Xgboost , LightGBM , CatBoost","Linear or Logistic Regression, Decision Trees ...",None of these activities are an important part...,"DataCamp, Udemy","Twitter (data science influencers), Kaggle (no..."


In [21]:
def multibinarize_columns(df, columns_to_encode):
    df_encoded = df.copy()
    
    for column in columns_to_encode:
        # Handle missing values by filling with 'None'
        df_encoded[column].fillna('None', inplace=True)
        
        #Ignore the space between long answers
        df_encoded[column] = df_encoded[column].apply(lambda x: x.replace(' ', ''))
        # Split the column into lists, not splitting words within '()'
        items_split = df_encoded[column].apply(lambda x: re.split(r',(?![^()]*\))', x))
        
        # Perform encoding using MultiLabelBinarizer
        mlb = MultiLabelBinarizer()
        items_encoded = mlb.fit_transform(items_split)
        
        # Create a DataFrame from the encoded items
        # Append the column name to make it unique for each column
        classes = [f"{column}_{cls}" for cls in mlb.classes_]
        items_encoded_df = pd.DataFrame(items_encoded, columns=classes)
        
        # Drop the original column from the DataFrame
        df_encoded.drop(column, axis=1, inplace=True)
        
        # Concatenate the new encoded DataFrame to the original DataFrame
        df_encoded = pd.concat([df_encoded, items_encoded_df], axis=1)
        
    return df_encoded

In [22]:
columns_to_binarize = list(substrings_new_columns.values())
# Exclude dropped columns from the categorical columns
filtered_columns = [col for col in columns_to_binarize if col not in null_columns]
df_encoded = multibinarize_columns(combined_dataset, filtered_columns)

In [23]:
df_encoded.to_csv('preprocess_encoded.csv',index= False)

In [24]:
df_encoded

,Age,Gender,Country,Education,Job,Years_of_programming,Years_of_machine_learning,Company_size,Yearly_compensation,Language_Bash,...,"Media_sources_Emailnewsletters(DataElixir,O'ReillyData&AI,etc)","Media_sources_JournalPublications(peer-reviewedjournals,conferenceproceedings,etc)","Media_sources_Kaggle(notebooks,forums,etc)",Media_sources_None,Media_sources_Other,"Media_sources_Podcasts(ChaiTimeDataScience,O’ReillyDataShow,etc)","Media_sources_Reddit(r/machinelearning,etc)","Media_sources_SlackCommunities(ods.ai,kagglenoobs,etc)",Media_sources_Twitter(datascienceinfluencers),"Media_sources_YouTube(KaggleYouTube,CloudAIAdventures,etc)"
0,32,Man,United States of America,Master’s degree,Data Engineer,5-10 years,1-2 years,"10,000 or more employees",112500.0,0,...,0,0,1,0,0,0,1,1,1,1
1,38,Man,Argentina,Bachelor’s degree,Software Engineer,10-20 years,MachineLearningEngineer,"1000-9,999 employees",17500.0,1,...,1,0,1,0,0,0,0,0,0,1
2,32,Man,United States of America,Master’s degree,Data Scientist,5-10 years,3-4 years,250-999 employees,137500.0,1,...,0,0,0,1,0,0,0,0,0,0
3,32,Man,Japan,Master’s degree,Software Engineer,3-5 years,NaN,NaN,NaN,0,...,0,0,0,1,0,0,0,0,0,0
4,32,Man,India,Bachelor’s degree,DataAnalyst,< 1 years,NaN,NaN,NaN,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33946,42,Man,United States of America,Master’s degree,Data Architect,10-20 years,Under 1 year,"10,000 or more employees",112500.0,1,...,0,0,1,0,0,0,0,0,1,1
33947,38,Man,India,Master’s degree,Data Scientist,1-3 years,1-2 years,"10,000 or more employees",2500.0,0,...,0,0,0,0,0,0,0,0,0,1
33948,42,Man,India,Bachelor’s degree,Data Engineer,1-3 years,MachineLearningEngineer,0-49 employees,500.0,0,...,0,0,0,0,0,0,0,0,0,1
33949,28,Man,Turkey,Master’s degree,Engineer (non-software),< 1 years,Under 1 year,"10,000 or more employees",27500.0,0,...,0,0,1,0,0,0,0,0,1,1


In [25]:
df_encoded['Yearly_compensation'].isnull().sum()

4089